## 전국 신규 민간 아파트 분양 가격 동향 자료 전처리 및 분석


1. 데이터 : 공공데이터 포탈에서 제공, 2015년 10월부터 2018년 7월까지의 주택분양 가격 정보 제공
    - 참고 -파일명 수정된 상태(한글파일 -> 영문과 숫자)
    - *한국 데이터셋 파일들 : 인코딩 수정 또는 확인(cp949=, euv-kt :  utg=8 형식으로 변환 및 재저장)
    - 단순 경고등이 발생(skip 명령어 적용)

In [1]:
# 단순 경고 메세지 skip 할 수 있게 하는 설정
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import re

2015년 1월~18년 7월까지의 신규 아파트 분양 정보 확인 및 전처리(평당 분양가 격등도 추가)

In [3]:
#avg18.07.csv 파일을 read: 변수명  pre_sale
pre_sale = pd.read_csv('C:/0.ITStudy/10.Pandas/aptPriceDataSet/avg18.07.csv', encoding='utf-8')
pre_sale

FileNotFoundError: File b'C:/0.ITStudy/10.Pandas/aptPriceDataSet/avg18.07.csv' does not exist

In [ ]:
'''
1. 5개 head
2. 5개 tail
3. null 보유 여부 확인
4. shape로 row수? column수?

'''
print(pre_sale.head())
print('---------------------------------------------------------')
print(pre_sale.tail())
print('---------------------------------------------------------')
print(pre_sale.isnull().sum())
print('---------------------------------------------------------')
print(pre_sale.isnull().any())
print('---------------------------------------------------------')
print(pre_sale.shape)
print('---------------------------------------------------------')

In [ ]:
# 각 컬럼별 type확인
pre_sale.dtypes

In [ ]:
# 분양가격에는 null값 보유. 따라서 null 개수를 제외한 유효한 데이터 개수 확인
pre_sale.info()

In [ ]:
pre_sale.isnull()

In [ ]:
# 분양가격 값만 검색해보기
pre_sale['분양가격(㎡)']

In [ ]:
# 분양가격으로 새로운 컬럼 도출(연산도 수행 가능한 타입으로 변환)
# raw 데이터는 유지, 새로운 컬럼 추가(분양가격, 평당 분양가격)
# 존재하는 컬럼으로 새로운 Series 객체 생성
pre_sale_price = pre_sale['분양가격(㎡)']
pre_sale_price

In [ ]:
# 분양가격 새로운 컬럼 생성(타입은 숫자 왜? 연산 수행)
# 숫자타입인 분양가격을 기반으로 평당 분양가격 컬럼 생성
# *3.3
'''
    errors='coerce' : 에러가 발생되어도 무시
    errors='raise' : 에러 발생 시 표현
'''
pre_sale['분양가격'] = pd.to_numeric(pre_sale_price, errors='coerce')

In [ ]:
pre_sale['분양가격'].dtype

In [ ]:
pre_sale_price.dtype

In [ ]:
# 불안정한 결측치가 nan이라는 명확한 표현으로 나타나짐
pre_sale.head(30)

In [ ]:
# 평당분양가격이라는 새로운 컬럼 생성해서 추가
# 분양가격*3.3 연산
pre_sale['평당분양가격'] = pre_sale['분양가격']*3.3
pre_sale.head(30)

In [ ]:
pre_sale.info()

In [ ]:
# 결측치를 단순 메모장에서 필수로 확인
# 분양가격, 평당분양가격 컬럼은 공백도 null로 집계

pre_sale['분양가격(㎡)'].isnull().sum()

In [ ]:
pre_sale['분양가격'].isnull().sum()

In [ ]:
pre_sale.describe()

In [ ]:
pre_sale.describe(include=[np.object])

In [ ]:
# 연도 기준으로 2017년 데이터 검색
pre_sale['연도']==2017
pre_sale_2017 = pre_sale[pre_sale['연도']==2017]

In [ ]:
pre_sale_2017.shape

In [ ]:
# 지역명 확인
# select distinct 지역명 form pre_sale;
pre_sale['지역명'].value_counts()

In [ ]:
# 전국 평균 분양 가격

# 연도별 수치 타입인 분양가격과 평균분양가격 집계정보 확인
pre_sale.groupby(pre_sale['연도']).describe()

In [ ]:
pre_sale.groupby(pre_sale['연도']).describe().T

In [ ]:
pre_sale.pivot_table

In [ ]:
'''
피벗 함수 이해를 위한 전처리
'''

In [ ]:
# 전용면적 60㎡이하인 데이터만 검색 
a = pre_sale.loc[pre_sale['규모구분'] == '전용면적 60㎡이하']
a

In [ ]:
a.shape

In [ ]:
b = a.loc[a['연도']==2015]
b

In [ ]:
b.shape

In [ ]:
b['평당분양가격'].mean()

In [ ]:
# 2015년도 데이터를 기준으로 '평당분양가격'을 '규모구분'과 '연도' 별로 평균값 구하기

In [ ]:
# pivot_table(values=None, index=None, columns=None)
pre_sale.pivot_table('평당분양가격', '규모구분', '연도')

규모 구분이 전체로 되어있는 금액으로 연도별 변동금액 알아보기

In [ ]:
all = pre_sale.loc[pre_sale['규모구분']=='전체']
all

In [ ]:
# 평당분양가격, 지역명, 연도 관련해서 pivot_table 
yearUp = all.pivot_table('평당분양가격', '지역명', '연도')
yearUp.reset_index()

In [ ]:
print(yearUp.dtypes)
yearUp['변동액'] = (yearUp[2018]-yearUp[2015])
yearUp

In [ ]:
# 소수 데이터들을 정수화 시킬 수 있는 설정 + 천단위 쉼표 처리 추가
pd.options.display.float_format = '{:,.0f}'.format

In [ ]:
yearUp

In [ ]:
'''
? 지역별 평당분양가격 합계 구하기
피벗의 인수값 : 평당분양가격, 규모구분, 
'''

In [ ]:
atable = pre_sale.pivot_table('평당분양가격', '규모구분', '지역명')
atable

In [ ]:
# avg15.09.csv로 DataFrame 객체 생성하기
df = pd.read_csv(r'C:\0.ITStudy\10.Pandas\aptPriceDataSet\avg15.09.csv', encoding='utf-8', header=0, skiprows=1)
df

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
year = df.iloc[0]
year

In [ ]:
month = df.iloc[1]
month

In [ ]:
'''
python의 enumerate() 함수의 메뉴얼 참조 필수

year와 month의 데이터를 결합해서 '2014년 4월' 형태로 가공

1. join()/ year[index]/ month[index]....
2. 2014 이후부터 2015 이전까지 2014로 채우기
3. 2015 이후는 2015로 채우기
'''

for i, y in enumerate(year):
    if i>2 and i<15:
        year[i] = ' '.join(['2014년', month[i]])
    elif i >= 15:
        year[i] = ' '.join(['2015년', month[i]])
    elif i == 2:
        year[i] = ' '.join([year[i],month[i]])
    elif i ==1:
        year[i] = '시구군'

year

In [ ]:
df = df.drop(1)
df

In [ ]:
df.columns = year
#df.drop(df.index[[0,1]], inplace=True)
df

In [ ]:
df = df.drop(0)

In [ ]:
df.head()

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

In [ ]:
del df['index']

In [ ]:
df.head()

In [ ]:
df.fillna("", inplace=True)
df.head()

In [ ]:
df.replace(df.iloc[2]['구분'], "", inplace=True)
df.head()

In [ ]:
df.replace(df.iloc[12]['구분'], '', inplace=True)

In [ ]:
df

In [ ]:
df['구분'] = df['구분']+df['시구군']

In [ ]:
del df['시구군']

In [ ]:
df

In [ ]:
for i, y in enumerate(year):
    if i>2 and i<15:
        year[i] = ' '.join(['2014년', month[i]])
    elif i >= 15:
        year[i] = ' '.join(['2015년', month[i]])
    elif i == 2:
        year[i] = ' '.join([year[i],month[i]])
    elif i ==1:
        year[i] = '시구군'